### Müllabfuhr - Wahl von 5 Pfaden

[Aufgabe](muellabfuhr.pdf) - [Lösungshinweise](muellabfuhrL.pdf)

In [204]:
# die Matrizen D und P werden als globale Variablen vorausgesetzt.
def dist(v1, v2):
    '''
    returns: kürzeste Distanz zwischen v1 und v2
    '''
    return D[v1][v2]

def sPath(v1, v2):
    '''
    returns: einen kürzesten Weg zwischen v1 und v2
    '''
    return getPath(P,v1,v2)

def distRund(e):
    '''
    Länge des kürzesten Rundwegs, der die Kante e enthält
    '''
    v1, v2 = e
    return dist(0,v1)+dist(v1,v2)+dist(v2,0)

def distMit(v1, v2, e):
    '''
    kürzeste Distanz zwischen v1 und v2, wenn dabei Kante e durchlaufen wird
    
    '''
    va, vb = e
    return dist(v1,va)+dist(va,vb)+dist(vb,v2)

def distErweiter(p, e):
    '''
    Länge des Pfades, wenn man p erweitert, so dass er durch e geht und dann wieder zurück zu 0.
    '''
    return 


def pathKanten(p):
    ''' set mit den normierten Kanten von p '''
    tmp = set()
    for i in range(len(p)-1):
        a, b = p[i], p[i+1]
        if a < b: tmp.add((a,b))
        else: tmp.add((b,a))
    return tmp  

def pathCost(p):
    '''
    returns: Kosten, um den Pfad p zu durchlaufen
    '''
    tmp = 0 
    for i in range(len(p)-1):
        a, b = p[i], p[i+1]
        tmp+=G[a][b]
    return tmp


def nextKante(E1, R):
    '''
    returns: die nächste Kante aus E1, die nicht in R ist.
    '''
    for i in range(len(E1)):
        if E1[i] in E1 and E1[i] not in R:
            return E1[i]
        
def verlaengertRueckweg(p, e):
    '''
    returns: True, wenn der Weg über die Kante e den Rückweg von p verlängert
    '''
    return dist(p[-1],0) < distMit(p[-1],0,e)

def aufRueckwegVon(pp, e):
    '''
    returns: Index des Pfads p in pp, bei dem die Kante e auf dem Rückweg liegt, ohne diesen
    zu verlängern
    '''
    for i in range(5):
        if not verlaengertRueckweg(pp[i],e):
            return i
    return -1
        
def schnellsterPfadZu(pp, e):
    '''
    returns: Index des Pfads, der e integriert und die kleinste Rundreisekosten verursacht
    '''
    best = None
    best_val = inf
    va, vb = e
    for i in range(5):
        val = pathCost(pp[i]) + dist(va, vb) + dist(vb, 0)
        if val < best_val:
            best_val = val
            best = i
    return best

def mergePaths(p1, p2):
    '''
    verbindet die Pfade p1 und p2 zu einem Gesamtpfad. Voraussetzung ist, dass
    der Endknoten von p1 der Anfangsknoten von p2 ist
    '''
    return p1[:-1]+p2
        
def erweitere(p,e):
    '''
    returns: die kürzeste Möglichkeit, den Pfad p um die Kante e zu erweitern.
    '''
    v1 = p[-1]
    va, vb = e
    if dist(v1,va) < dist(v1,vb):
        return p[:-1] + sPath(v1,va) + (vb,)
    else:
        return p[:-1] + sPath(v1,vb) + (va,)
    
def pathKanten(p):
    ''' returns set mit den normierten Kanten von p '''
    tmp = set()
    for i in range(len(p)-1):
        a, b = p[i], p[i+1]
        if a < b: tmp.add((a,b))
        else: tmp.add((b,a))
    return tmp  
        
def updateE1(pp,E1):
    alleKanten = set()
    for p in pp:
        alleKanten = alleKanten | pathKanten(p)
    result = []
    for i in range(len(E1)):
        if E1[i] not in alleKanten:
            result.append(E1[i])
    print('updateE1',result)
    return result

def updateRR(pp):
    result = set()
    for p in pp:
        rueckweg = spath(p[-1],0)
        result = result | pathKanten(rueckweg)
    return result

def goHome(pp):
    for i in range(5):
        p = pp[i]
        pp[i] =  mergePaths(p,sPath(p[-1],0))
        
         
def pathCost(p):
    tmp = 0 
    for i in range(len(p)-1):
        a, b = p[i], p[i+1]
        tmp+=G[a][b]
    return tmp

def cost(pp):    
    max_cost = 0
    for p in pp:
        if pathCost(p)> max_cost:
            max_cost = pathCost(p)
    return max_cost     
        
         
    


In [244]:
%%time
'''
6: 190 ms, 7: 22.6 s, 8: 3min 3s     
'''
inf = float('inf')
nr = 7
f = open('./beispieldaten/muellabfuhr'+str(nr)+'.txt')
anzV, anzE = [int(x) for x in f.readline().split()]

G = [[inf]*anzV for _ in range(anzV)]
V = set()
E = set()
for i in range(anzE):
    von, bis, km = [int(x) for x in f.readline().split()]
    G[von][bis]=km
    G[bis][von]=km
    V.add(von)
    V.add(bis)
    
    if von < bis:
        E.add((von,bis))
    else:
        E.add((bis,von))
for i in range(anzV):
    G[i][i]=0
    
def floyd(c):
    n = len(c)
    d = [[0]*n for j in range(n)]   # Matrix D
    p = [[0]*n for j in range(n)]   # Matrix P
    for i in range(n):
        for j in range(n):
            d[i][j] = c[i][j]
            p[i][j] = i

    for k in range(n):
        for i in range(n):
            for j in range(n):
                tmp = d[i][k] + d[k][j]
                if tmp < d[i][j]:
                    d[i][j] = tmp
                    p[i][j] = p[k][j]
    return d, p

def getPath(p, i, j):
    if i == j: return (i,)
    return getPath(p,i,p[i][j]) + (j,)

def printMatrix(a):
    for i in range(len(a)):
        for j in range(len(a)):
            if a[i][j] == inf:
                print("{:4}".format("."),end='')
            else:
                print("{:<4}".format(a[i][j]),end='')
        print()
    print()
    
D, P = floyd(G)


CPU times: total: 18.9 s
Wall time: 18.9 s


In [245]:
# Berechne für jede Kante den kürzesten Rundweg, sortiere absteigend sortiert.
E1 = sorted(list(E),key = lambda e : -distRund(e))  
#print('E1',E1,len(E1))

# Initialisiere 5 Pfade, von 0 ausgehend
pp = [(0,),(0,),(0,),(0,),(0,)]     # Die 5 Tupel mit den Pfaden

# Initialisiere R die Menge der Kanten, die für einen Rückweg vorgesehen sind als leere Menge
R = set()
 

In [246]:
# Wähle in den noch zu überdeckenden Kanten die Kante mit dem längsten Rundweg, die noch nicht
#     für einen Rückweg vorgesehen ist.
while E1:
    en = nextKante(E1,R)
    if en is None:
        break
    #print('nächste Kante',en)

    '''
    Prüfe, ob es einen Pfad gibt, bei der die zu überdeckende Kante die Länge des kürzesten Rückwegs nicht        
      verlängert.
    Wenn ja, dann erweitere diesen Pfad um den Weg zur Kante (inklusive Kante)
    Wenn nein, dann selektiere den Pfad, der den kürzesten Rundweg hat, wenn die zu überdeckende Kante in den
      Rundweg integriert wird und erweiter diesen Pfad um den Weg zur Kante (inklusive Kante)
    '''
    if aufRueckwegVon(pp,en) != -1:
        ip = aufRueckwegVon(pp,en)
        #print('auf Rückweg von Pfad',ip)
    else:
        ip = schnellsterPfadZu(pp,en)
        #print('schnellster Pfad',ip)

    pp[ip] = erweitere(pp[ip],en)    
    #print('nach Pfaderweiterung',pp)

    for e in pathKanten(pp[ip]):
        if e in E1:
            E1.remove(e)
    #print('update E1', E1, len(E1))

    R = set()
    for p in pp:
        for e in pathKanten(sPath(p[-1],0)):
            if e in E1:
                R.add(e)

    #print('build R',R)


In [247]:
goHome(pp)
for p in pp:
    print(p,pathCost(p))
cost(pp)

(0, 1, 3, 6, 11, 13, 45, 300, 155, 157, 248, 428, 185, 229, 185, 428, 241, 479, 155, 355, 45, 238, 306, 351, 367, 351, 236, 238, 45, 13, 99, 472, 152, 145, 152, 201, 152, 320, 152, 339, 364, 129, 13, 264, 259, 199, 264, 281, 275, 496, 190, 281, 496, 281, 146, 264, 13, 99, 123, 364, 129, 226, 286, 264, 13, 125, 85, 123, 102, 462, 129, 99, 13, 11, 6, 159, 338, 8, 53, 153, 168, 381, 266, 150, 314, 266, 314, 212, 117, 70, 494, 432, 161, 296, 106, 168, 153, 56, 161, 263, 153, 53, 8, 338, 159, 6, 11, 60, 156, 425, 476, 216, 32, 425, 216, 51, 425, 51, 288, 156, 262, 92, 475, 156, 475, 76, 208, 31, 76, 60, 92, 60, 11, 91, 208, 76, 373, 101, 11, 6, 18, 442, 107, 332, 392, 74, 107, 480, 192, 107, 480, 180, 469, 454, 442, 452, 180, 73, 391, 109, 441, 391, 276, 180, 73, 439, 441, 18, 6, 3, 1, 4, 17, 33, 34, 44, 93, 291, 222, 483, 221, 114, 230, 222, 230, 114, 291, 424, 349, 292, 181, 349, 444, 426, 291, 93, 253, 186, 490, 270, 39, 490, 39, 345, 380, 426, 291, 93, 39, 270, 44, 39, 34, 33, 17, 4, 1,

2933896

#### Beispieldaten

In [82]:
nr = 0
f = open('./beispieldaten/muellabfuhr'+str(nr)+'.txt')
print(f.read())

10 13
0 2 1
0 4 1
0 6 1
0 8 1
1 2 1
2 3 1
3 4 1
4 5 1
5 6 1
6 7 1
7 8 1
8 1 1
8 9 1


In [101]:
def pathDist(p,k):
    '''
    die geringsten Gesamtkosten, wenn vom Ende des Pfades p ein Ende der Kante k erreicht werden soll
    (die Kante wird noch nicht durchlaufen)
    '''
    a,b = k
    v = p[-1] 
    return pathCost(p)+min(D[v][a],D[v][b])

def zurKante(p,k):
    ''' kürzeste Pfad vom Pfad p zur Kante k, mit Durchlaufe der Kante '''
    a, b = k
    v = p[-1]
    p0 = list(p)[:-1]
    ziel = a
    if D[v][a] < D[v][b]:
        return tuple(p0)+getPath(P,v,a)+(b,)
    else:
        return tuple(p0)+getPath(P,v,b)+(a,)
        
def pathKanten(p):
    ''' set mit den normierten Kanten von p '''
    tmp = set()
    for i in range(len(p)-1):
        a, b = p[i], p[i+1]
        if a < b: tmp.add((a,b))
        else: tmp.add((b,a))
    return tmp   

def pathCost(p):
    tmp = 0 
    for i in range(len(p)-1):
        a, b = p[i], p[i+1]
        tmp+=G[a][b]
    return tmp

def cost(pp):    
    max_cost = 0
    for p in pp:
        if pathCost(p)> max_cost:
            max_cost = pathCost(p)
    return max_cost


In [102]:
pp = [(0,),(0,),(0,),(0,),(0,)]     # Die 5 Tupel mit den Pfaden
E1 = E.copy()              # die noch zu überdeckenden Kanten
 
zaehl = 0
while len(E1)>0:
    #print('Pfade zu Beginn',pp)
    
    # das Gewicht der Kante (a,b) ist der kürzeste Weg nach a + Gewicht der Kante von a nach b +
    # kürzester Weg von b nach a
    gewichte = {(a,b): D[0][a]+G[a][b]+D[b][0] for a,b in E1}   
    
    reihenfolge = sorted(E1,key = lambda x : -gewichte[x])
    kante  = reihenfolge[0]
    
    #print('gewählte Kante / Kantengewicht',kante, gewichte[kante])
    dd = [pathDist(p,kante) for p in pp].copy()
    #print('Pathdistanzen zur gewählten Kante',dd)
    i = dd.index(min(dd))  
    #print('minindex',i)
    path = pp[i] 
    
    p1 = zurKante(path,kante)
    #print('zur Kante',p1)
    
    pp[i] = p1
    #print('*',pathKanten(p1))
    #print('Kanten bisher',len(E1))
    E1 = E1 - pathKanten(p1).copy()
    #print('Kanten jetzt',len(E1))
    zaehl+=1
    #print(zaehl)
    #print('--------------')

# jetzt auf dem kürzesten Weg zurück
for i in range(5):
    p = pp[i]
    v = p[-1]
    p1 = tuple(list(p[:-1]))
    pp[i] = p1 + getPath(P,v,0)
    
for p in pp:
    print(pathCost(p),p)
print("Maximaler Weg:",cost(pp))

18 (0, 6, 3, 2, 3, 6, 0)
16 (0, 6, 7, 4, 0)
15 (0, 6, 3, 4, 0)
19 (0, 6, 7, 5, 4, 5, 3, 6, 0)
23 (0, 6, 3, 1, 6, 0, 5, 7, 6, 0)
Maximaler Weg: 23


-----

-------